Intermodal nx_graph can be loaded from OSM [example](graph_generator.ipynb)

In [ ]:
from shapely import Point
from objectnat import get_accessibility_isochrones,GraphType
from objectnat import get_intermodal_graph_from_osm
graph = get_intermodal_graph_from_osm(city_osm_id=1114252, city_crs=32636)

In [ ]:
import geopandas as gpd
points = gpd.GeoSeries([Point(347111,6647736),Point(347611,6647336),Point(347612,6647336)])
accessibility_isochrones, public_transport_routes, public_transport_stops = get_accessibility_isochrones(
    graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK],
    points=points,
    weight_value=10,
    weight_type="time_min",
    city_graph=graph,
    city_crs=32636,
)
m1 = accessibility_isochrones.explore(column = 'point_number',cmap = "viridis_r",categorical=True)
public_transport_routes.explore(m=m1,color='red') # if not None
public_transport_stops.explore(m=m1,color='purple')
points.explore(m=m1)# if not None
m1